<a href="https://colab.research.google.com/github/Baran-92/cyberwvu_demo/blob/main/VideoSplitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Split one uploaded video into 10 equal parts (Colab-ready) ---

# 0) Upload the video
from google.colab import files
up = files.upload()                            # pick your video file
VIDEO_PATH = next(iter(up.keys()))
print("Using:", VIDEO_PATH)

# 1) Settings
N_PARTS   = 10          # number of output clips
REENCODE  = False       # False = fastest (stream copy), True = safest (re-encode)

# 2) Utilities
import os, json, shutil, subprocess

def ffprobe_duration(path):
    # Get precise duration via ffprobe (ships with Colab)
    out = subprocess.check_output([
        "ffprobe","-v","error","-select_streams","v:0",
        "-show_entries","format=duration","-of","json", path
    ]).decode()
    return float(json.loads(out)["format"]["duration"])

# 3) Compute segment length
dur = ffprobe_duration(VIDEO_PATH)
seg = dur / N_PARTS
print(f"Duration: {dur:.3f} s  →  segment ≈ {seg:.3f} s")

# 4) Make output folder
outdir = "splits"
os.makedirs(outdir, exist_ok=True)

# 5) Split loop
for i in range(N_PARTS):
    start = i * seg
    out_f = os.path.join(outdir, f"part_{i+1:02d}.mp4")
    if i < N_PARTS - 1:
        if REENCODE:
            cmd = [
                "ffmpeg","-y","-ss", f"{start:.3f}","-i", VIDEO_PATH,
                "-t", f"{seg:.3f}",
                "-c:v","libx264","-preset","veryfast","-crf","23",
                "-c:a","aac","-b:a","128k", out_f
            ]
        else:
            cmd = [
                "ffmpeg","-y","-ss", f"{start:.3f}","-i", VIDEO_PATH,
                "-t", f"{seg:.3f}","-c","copy", out_f
            ]
    else:
        # Last part: go to end (no -t) so rounding never truncates
        if REENCODE:
            cmd = [
                "ffmpeg","-y","-ss", f"{start:.3f}","-i", VIDEO_PATH,
                "-c:v","libx264","-preset","veryfast","-crf","23",
                "-c:a","aac","-b:a","128k", out_f
            ]
        else:
            cmd = [
                "ffmpeg","-y","-ss", f"{start:.3f}","-i", VIDEO_PATH,
                "-c","copy", out_f
            ]
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)

print("\nDone. Files are in ./splits/")
# 6) Optional: zip for one-click download
shutil.make_archive("splits", "zip", outdir)
files.download("splits.zip")


Saving Media1.mp4 to Media1.mp4
Using: Media1.mp4
Duration: 68.820 s  →  segment ≈ 6.882 s
Running: ffmpeg -y -ss 0.000 -i Media1.mp4 -t 6.882 -c copy splits/part_01.mp4
Running: ffmpeg -y -ss 6.882 -i Media1.mp4 -t 6.882 -c copy splits/part_02.mp4
Running: ffmpeg -y -ss 13.764 -i Media1.mp4 -t 6.882 -c copy splits/part_03.mp4
Running: ffmpeg -y -ss 20.646 -i Media1.mp4 -t 6.882 -c copy splits/part_04.mp4
Running: ffmpeg -y -ss 27.528 -i Media1.mp4 -t 6.882 -c copy splits/part_05.mp4
Running: ffmpeg -y -ss 34.410 -i Media1.mp4 -t 6.882 -c copy splits/part_06.mp4
Running: ffmpeg -y -ss 41.292 -i Media1.mp4 -t 6.882 -c copy splits/part_07.mp4
Running: ffmpeg -y -ss 48.174 -i Media1.mp4 -t 6.882 -c copy splits/part_08.mp4
Running: ffmpeg -y -ss 55.056 -i Media1.mp4 -t 6.882 -c copy splits/part_09.mp4
Running: ffmpeg -y -ss 61.938 -i Media1.mp4 -c copy splits/part_10.mp4

Done. Files are in ./splits/


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>